In [1]:
import json
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder

IMG_FEAT_PATH = '../data/simaese_img_feats/simaese_mob_299_sim_best_train_loss.pkl'

fin = open(IMG_FEAT_PATH, 'rb')
train_feats = pickle.load(fin)
fin.close()
print(train_feats[0][0], train_feats[0][1], train_feats[0][2].shape)

w_ebb99e5 ../data/train/0ba0e60f3.jpg (256,)


In [2]:
# only keep label cnt >= 2
x_data, y_data = [], []
train_cnt = {}
for label, _, _ in train_feats:
    if label not in train_cnt:
        train_cnt[label] = 0
    train_cnt[label] += 1

filter_labels = set()
for label in train_cnt:
    if train_cnt[label] >= 3:
        filter_labels.add(label)
print('left labels', len(filter_labels))

for label, _, feat in train_feats:
    if label in filter_labels:
        x_data.append(feat)
        y_data.append(label)

le = LabelEncoder()
y_l_data = le.fit_transform(y_data)
x_data = np.array(x_data)
print(x_data.shape, y_l_data.shape)

left labels 1646
(11054, 256) (11054,)


In [3]:
print(le.classes_)
print(len(le.classes_))

['w_0027efa' 'w_003bae6' 'w_007fefa' ... 'w_fec5547' 'w_ff2157c'
 'w_ffa542b']
1646


In [4]:
# cv check
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(
    x_data, y_l_data, test_size=0.33, random_state=42)
lr_model = LogisticRegression(
    random_state=0, solver='lbfgs', multi_class='multinomial')
lr_model.fit(x_data, y_l_data)
print('fit done')

fit done


In [5]:
print('all data score', lr_model.score(x_data, y_l_data))
print('test data score', lr_model.score(X_test, y_test))
all_pred = lr_model.predict_proba(x_data)
test_pred = lr_model.predict_proba(X_test)
print(test_pred[0], sum(test_pred[0]))


def check_topk_acc(y_true, y_pred, k=3):
    correct = 0
    for i, tmp_pred in enumerate(y_pred):
        topk = tmp_pred.argsort()[-k:]
        # print(topk,y_true[i])
        if y_true[i] in topk:
            correct += 1
    print(correct*1.0/len(y_true))


print('check topk for distribution')
check_topk_acc(y_test, test_pred, 3)
check_topk_acc(y_test, test_pred, 4)
check_topk_acc(y_test, test_pred, 5) 

# label_cnt >=4 
# all data score 0.24545454545454545
# test data score 0.24692158133506156
# [3.61351534e-06 1.42040442e-06 1.19540025e-04 ... 4.44171080e-06
#  6.37715558e-08 3.44947124e-08] 1.0000000000000007
# check topk for distribution
# 0.40926766040181467
# 0.4594944912508101
# 0.49773169151004537

# label_cnt >= 3
# all data score 0.20915505699294373
# test data score 0.21134868421052633
# [8.09095308e-04 4.02344437e-05 5.46170862e-03 ... 3.85406255e-06
#  5.62370183e-07 4.86558397e-06] 1.0
# check topk for distribution
# 0.3519736842105263
# 0.39473684210526316
# 0.4322916666666667


all data score 0.20915505699294373
test data score 0.21134868421052633
[8.09095308e-04 4.02344437e-05 5.46170862e-03 ... 3.85406255e-06
 5.62370183e-07 4.86558397e-06] 1.0
check topk for distribution
0.3519736842105263
0.39473684210526316
0.4322916666666667


In [6]:
with open('../data/checkpoints/lr_top3_model.pkl', 'wb') as fout:
    pickle.dump(lr_model, fout)
print('save done')

save done
